pip install spacy

ściągnąć polski model z 
http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=view&target=pl_spacy_model-0.1.0.tar.gz
i zainstalować: python -m pip install PATH/TO/pl_spacy_model-x.x.x.tar.gz

pliki json z rzeczownikami pobrane z:
https://github.com/tombusby/PolishCaseTrainer/tree/master/polish_case_trainer/word/word-data

## Ładowanie modułów

In [4]:
import requests
import bs4
import spacy
import re
import random
from collections import defaultdict

In [5]:
nlp = spacy.load('pl_spacy_model')
from spacy.lang.pl.stop_words import STOP_WORDS

In [769]:
sample_page = 'https://pl.wikipedia.org/wiki/Psychoza'

## Utworzenie stałych

In [12]:
CODE_DICT = {'ą':('\u0105', '%C4%85','\\xc4\\85'), 'ę':('\u0119', '%C4%99', '\\xc4\\99'), 'ł':('\u0142', '%C5%82','\\xc5\\82'), 'ś':('\u015b', '%C5%9B','\\xc5\\9b'),
            'ć':('\u0107', '%C4%87', '\\xc4\\87'), 'ń':('\u0144', '%C5%84','\\xc5\\84'), 'ó':('\u00f3', '%C3%B3','\\xc3\\xb3'), 'ź':('\u017a', '%C5%BA','\\xc5\\ba'), 
             'ż':('\u017c', '%C5%BC','\\xc5\\bc'), 'Ł':('\u0141', '%C5%81', '\\xc5\\81'), 'Ś':('\u015a', '%C5%9A','\\xc5\\9a'), 'Ć':('\u0106', '%C4%86', '\\xc4\\86'), 
             'Ź':('\u0106', '%C4%86','\\xc4\\86'), 'Ż':('\u017b', '%C5%Bb', '\\xc5\\bb')}

In [13]:
przypadki = ['mianownik', 'dopełniacz', 'celownik', 'biernik', 'narzędnik', 'miejscownik', 'wołacz']

In [14]:
DECLINATION_FORM = {'accusative' : 'biernik', 'instrumental' : 'narzędnik', 'dative' : 'celownik', 'locative' : 'miejscownik', 'genitive' : 'dopełniacz', 
                    'nominative' : 'mianownik', 'vocative' : 'wołacz'}

## Ładowanie treści

In [773]:
def load_wiki_page(url):
    # Generator służący do ładowania strony
    res = requests.get(url)
    res.raise_for_status()
    wiki = bs4.BeautifulSoup(res.text, "html.parser")
    for _ in wiki.select('p'): #Bierzemy to co jest zawarte między tagami <p> </p>
        yield _.getText()


def prepare_text(brackets="[]"):
    # Buduwanie stringa z generatora i usuwanie nowych linii i no break-space
    res = []
    for _ in load_wiki_page(sample_page):
        res.append(_.replace('\n','').replace('\xa0',''))
    tekst = ''.join(res)
    
    # Usuwanie wszystkiego co znajduje się między nawiasami [], np. wszystkie przypisy typu: wyraz[14]
    count = [0] * (len(brackets) // 2)
    saved_chars = []
    for character in tekst:
        for i, b in enumerate(brackets):
            if character == b: 
                kind, is_close = divmod(i, 2)
                count[kind] += (-1)**is_close 
                if count[kind] < 0: 
                    count[kind] = 0 
                else:  
                    break
        else: 
            if not any(count): 
                saved_chars.append(character)
    return ''.join(saved_chars)

tekst = prepare_text()

### Zamiana tekst na nlp i utworzenie zdań

In [774]:
doc = nlp(tekst)
sentences= [sentence for sentence in doc.sents]

### Funkcja oczyszczająca słowa z tekstu

In [775]:
def clean_upN(text, inc_dot = False):
    # Potrzebna dla funkcji 'Najczęściej używane słowa'
    # usuwa przecinki, kropki, niepotrzebne spacje
    from string import digits, punctuation
    if inc_dot: # Czy kropka (.) ma być uwzględniona
        p = punctuation.replace('.','') + '—…' 
    else:
        p = punctuation + '—…'
    irr = len(digits + p)
    change = text.maketrans(digits + p, ' '*irr)
    result = text.translate(change)
    return result.rstrip().lstrip()

words_clean = [clean_upN(word) for word in tekst.split()]

### Najczęściej używane słowa w tekście

In [776]:
def most_common_words_func(n, output_words_only = False):
    from spacy.lang.pl.stop_words import STOP_WORDS
    from string import punctuation
    
    accurance = {}
    for word in words_clean:
        if word.lower() not in STOP_WORDS and word.lower() not in punctuation +'–':
            accurance[word] = accurance.get(word,0) +1
    
    #sortujemy słowa względem popularności
    accurance_sort = [(word, acc) for word, acc in sorted(accurance.items(), key = lambda item: item[1], reverse=True) if word !=''][:n]
    
    #czy funkcja ma zwrócić same słowa, czy słowa wraz z częstotliwościami wystąpień
    if output_words_only:
        return [item[0] for item in accurance_sort]
    else:
        return accurance_sort
    
most_common_words = most_common_words_func(50, True)

### Stworzenie sekwencji par słów bazujących ma łańcuchu Marcova

In [777]:
def make_pairs_gen(words_clean):
    # Generator pary krotek: (wyraz pierwszy, wyraz drugi), 
    # (wyraz drugi, wyraz trzeci) itp...
    for i in range(len(words_clean)-1): 
        yield (words_clean[i], words_clean[i+1]) 

def make_pairs(treshold = 3):
    from collections import Counter
    pairs = make_pairs_gen(words_clean)
    
    # Jeżeli słowa nie było w słowniku, dodaj je jako klucz
    # jeżeli zaś już było rozszerz jego value (listę) o kolejny element 
    pairs_dict = {}
    for w1, w2 in pairs:
        if w1 in pairs_dict.keys():
            pairs_dict[w1].append(w2)
        else:
            pairs_dict[w1] = [w2]
    
    # Utwórz słownik, który będzie przechowywał zliczenia słów
    pairs_dict_app = dict()
    for key in pairs_dict.keys():
        pairs_dict_app[key] = dict(Counter(pairs_dict[key]))
    
    # Jeżeli słowo (value) wystąpiło więcej lub tyle samo razy co zadany treshold i nie należy ono, ani
    # jego para (value - element listy) do  STOP_WORD, wtedy dodaj je wraz z parą do nowego słownika
    pairs_sort = dict()
    for key, val in pairs_dict_app.items():
        w, n = [(word, val) for word, val in sorted(pairs_dict_app[key].items(), key = lambda item: item[1], reverse = True)][0]
        if n >=treshold and w != '' and key not in STOP_WORDS and w not in STOP_WORDS:
            pairs_sort[key] = w
    return pairs_sort

pairs_sort = make_pairs(3)

In [778]:
pairs_sort

{'stanie': 'psychozy'}

### Łączenie par key i value słownika w jedno

In [779]:
def glue_dict():
    return {(k +' ' + str(v)) for (k, v) in pairs_sort.items()}
sequences = glue_dict()

In [780]:
sequences

{'stanie psychozy'}

### Stworzenie podmiotów (enteties)

In [781]:
def gen_ents():
    # Zwraca słownik: entsD = {rodzaj_podmiotu: nazwa podmiotu}
    # np. {'geogName': 'Afryka'}
    entsD = defaultdict(set)
    for token in doc.ents:
        entsD[token.label_].add(token.text)
    return entsD

ents_dict = gen_ents()

def gen_ents_dict():    
    ents = list()
    for v in ents_dict.values():
        ents.append(v) # Tworzy listę setów z genEnts() bez nazw
    ents_full = set(y for x in ents for y in x) # Wrzuca wszystkie podmioty do jednego seta
    return ents, ents_full 

ents, ents_full = gen_ents_dict()

In [782]:
ents_dict

defaultdict(set,
            {'persName': {'Ernst von Feuchtersleben', 'Levine', 'Zigler'},
             'date': {'1845'},
             'geogName': {'Księżycu'},
             'placeName': {'Gdyni', 'Krakowie'}})

### Poprawianie ents_dict

In [783]:
for k,v in ents_dict.items():
    if k == 'geogName':
        gN = {elem for elem in v if elem[0].isupper()}
    elif k == 'persName':
        pN = {elem for elem in v if elem[0].isupper()}
    elif k == 'orgName':
        oN = {elem for elem in v if elem[0].isupper()}
    elif k == 'placeName':
        plN = {elem for elem in v if elem[0].isupper()}

if 'geogName' in ents_dict.keys():
    ents_dict['geogName'] = gN
if 'persName' in ents_dict.keys():
    ents_dict['persName'] = pN
if 'orgName' in ents_dict.keys():
    ents_dict['orgName'] = oN
if 'placeName' in ents_dict.keys():
    ents_dict['placeName'] = plN

In [784]:
ents_dict

defaultdict(set,
            {'persName': {'Ernst von Feuchtersleben', 'Levine', 'Zigler'},
             'date': {'1845'},
             'geogName': {'Księżycu'},
             'placeName': {'Gdyni', 'Krakowie'}})

### Funkcja wstawiająca puste miejsce

In [785]:
def replace_word_in_sent(word, sentence):
    word = str(word)
    sentence = str(sentence)
    newSent = re.compile(re.escape(word), re.IGNORECASE)
    return newSent.sub('_______________', sentence)

### Funkcja zwracająca wszystko co jesteśmy w stanie zidentyfikować (zdefiniowane grupy wyrazów, słowa częste i podmioty)

In [786]:
def find_good_word(sent):
    seq_list = [word for word in sequences if word  in str(sent)] # Zidentyfikowane grupy wyrazów (Marcov)
    mcw_list = [word for word in most_common_words if word  in str(sent)] # Słowa częste
    ents_list = [word for word in ents_full if word  in str(sent)] # Enteties (podmioty)
    return seq_list + mcw_list + ents_list
    

### Funkcja zwracająca nazwę słownika z enteties, do którego należy dane słowo

In [787]:
def return_label_based_on_word(word):
    if word in ents_full:
        for key in ents_dict.keys():
            for elem in ents_dict[key]:
                if elem == word:
                    return key
    else:
        return ''

### Funkcja znajdująca niepoprawne odpowiedzi

In [788]:
def write_bad_words_for_single(word, all_candidates = False):
    # Jeżeli all_candidates = True, zwrócone będą również kandydaci, które algorytm brał pod uwagę
    # ale ich nie uwzględnił
    sl = dict()
    all_c = []
    dict_name = return_label_based_on_word(word) # Sprawdzenie do jakiego słownika należy słowo
    if dict_name != '':
        lista = []
        while len(lista) <3: # Dopóki nie wygenerujemy 3 alternatywnych odpowiedzi...
            cand = random.sample(ents_dict[dict_name],1)[0] # losuj dowolne słowo z tego samego słownika
            all_c.append(cand) # i dodaj je do listy
            if same_letters(cand, word) and different_lemma(cand, word): ###jeżeli pierwsze litery naszego słowa i kandydata
                # się zgadzają oraz mają różną lemmę...
                lista.append(cand) # dodaj tego kandydata do listy
        sl[word] = list(set(lista)) # unikamy wielokrotnych powtórzeń tych samych słów alternatywnych
    else:
        sl[word] = ()
    if all_candidates:
        return sl, all_candidates
    else:
        return sl

### Funkcja upewniająca się, czy nie mamy do czynienia z tym samym słowem, tylko odmienionym w inny sposób

In [789]:
def different_lemma(word1, word2):
    # np. dla different_lemma('związkowi radzieckiemu', 'związku radzieckiego')
    # False
    doc1 = nlp(word1)
    doc2 = nlp(word2)
    res1 = []; res2 = []
    for token in doc1:
        res1.append((token.lemma_))
    for token in doc2:
        res2.append((token.lemma_))
    return res1 != res2

### Funkcja badająca czy słowa zaczynają się tą samą literą

In [790]:
def same_letters(word1, word2):
    return word1[0].islower() == word2[0].islower() or word1[0].isupper() == word2[0].isupper()

### Funkcja zwracająca złe słowa (alternatywy) dla wskazanego dobrego

In [791]:
def write_bad_words(good_words, sort_ = True):
    sl = []
    for word in good_words: #dla każdego słowa w dobrych słowach
        sl.append(write_bad_words_for_single(word)) # utwórz słownik {słowo: [lista_alternatywnych_slow]} i dodaj go do kolekcji sl
    sort_sl = dict(pair for d in sl for pair in d.items()) # Utwórz z sl jeden słownik

    def sort_by_values_len(dict):
        # Posortuj słownik w zależności od długości [listy_alternatywnych_slow]
        dict_len= {key: len(value) for key, value in dict.items()}
        import operator
        sorted_key_list = sorted(dict_len.items(), key=operator.itemgetter(1), reverse=True)
        sorted_dict = [{item[0]: dict[item [0]]} for item in sorted_key_list]
        return sorted_dict
    
    # czy ma być zwrócona lista słowników {słowo: [lista_alternatywnych_slow]}  w formie posortowanej czy nie
    if sort_:
        return sort_by_values_len(sort_sl)
    else:
        return sl

### Funkcja losująca zdanie

In [792]:
def pick_sentence(best = True):
    found = False
    avoid = ['w.', 'w. ', ' r', 'r', 'r.', 'r ', 'm', ' m', 'm ', 'np', 'np.', 'K', 'C'] #unikaj tych 
    while not found: # Dopóki nie znaleziono właściwego zdania
        los = random.randint(0,len(sentences)) # Losuj dowolną liczbę z ilości dostępnych zdać
        wybrane_zdanie = sentences[los] # Wylosuj zdanie
        good_words = find_good_word(wybrane_zdanie) # Znajdź w nim dobre słowa (kandydatów)
        good_words = [w for w in good_words if w not in avoid]
        #Jeżeli zdanie zaczyna się wielką literą i nie jest za krótkie i ma co najmniej 3 słowa kandydujące...
        if len(good_words)>=3 and len(str(wybrane_zdanie).split()) >5 and str(wybrane_zdanie)[0].isupper():
            
            # napisz dla tych słów kandydujących słowa alternatywne
            bad_words = write_bad_words(good_words)
            if best:
                # jeżeli dane słowo kandydujące ma co najmniej 2 alternatywy
                if sum(map(len, bad_words[0].values())) >=2:
                    found = True
            else:
                found = True
    print(bad_words)
    
    #zwróć wybrane zdanie, wybrane słowo i numer zdania
    return wybrane_zdanie, ''.join(bad_words[0].keys()), los

picked_sentence, chosen_word, los = pick_sentence(best=False)

[{'zaburzeń': ()}, {'świadomości': ()}, {'umysłu': ()}, {'osoba': ()}, {'Psychoza': ()}, {'psychiatrii': ()}, {'stan': ()}, {'zakłóceń': ()}, {'percepcji': ()}]


In [793]:
picked_sentence

Psychoza (gr. psyche – dusza i osis – szaleństwo) – zaburzenie psychiczne definiowane w psychiatrii jako stan umysłu, w którym na skutek zakłóceń w postrzeganiu (percepcji) rzeczywistości, a czasem także zaburzeń świadomości, osoba ma trudność z odróżnieniem tego, co jest rzeczywiste, a co nie.

In [794]:
chosen_word

'zaburzeń'

In [795]:
if (los-1) >=0:
    print(sentences[los-1])
replace_word_in_sent(chosen_word, picked_sentence)

'Psychoza (gr. psyche – dusza i osis – szaleństwo) – zaburzenie psychiczne definiowane w psychiatrii jako stan umysłu, w którym na skutek zakłóceń w postrzeganiu (percepcji) rzeczywistości, a czasem także _______________ świadomości, osoba ma trudność z odróżnieniem tego, co jest rzeczywiste, a co nie.'

In [796]:
answers = write_bad_words_for_single(chosen_word)

In [797]:
answers

{'zaburzeń': ()}

# ---Poniższe potrzebne do przypadku gdy nie znajdzie się alternatywnych odpowiedzi---

# Deklinacja (rozwiązanie sieciowe)

### Funkcja znajdująca lemmę słowa

In [6]:
def ret_lemma(word):
    doc = nlp(word)
    for token in doc:
        return str(token.lemma_)

### Zmień polskie znaki diakrytyczne na znaki w systemie unicode lub unicode UTF-8

In [7]:
def change_to(word, coding):
    if coding=='unicode':
        word.replace('ą','\\xc4\\x85').replace('ę','\\xc4\\x99').replace('ł','\\xc5\\82').replace('ś','\\xc5\\9b').replace('ć','\\xc4\\87').replace('ń','\\xc5\\84').replace('ó','\\xc3\\xb3').replace('ź','\\xc5\\ba').replace('ż','\\xc5\\bc')
    if coding=='UTF-8':
        for k,v in CODE_DICT.items():
            word = word.replace(k, v[1])
    return word

### Zmień znaki w systemie unicode UTF-8 na polskie diakrytyczne

In [8]:
def change_from(encoded_word): ###USUNĄĆ
    if '%' in encoded_word: #UTF-8
        for k,v in CODE_DICT.items():
            encoded_word = encoded_word.replace(v[1],k)
    return encoded_word
    

### Popraw znaki na polskie litery

In [9]:
def correct_word(word):
    return word.replace('\\xc4\\x85','ą').replace('\\xc4\\x99','ę').replace('\\xc5\\x82','ł').replace('\\xc5\\x9b','ś').replace('\\xc4\\x87','ć').replace('\\xc5\\x84','ń').replace('\\xc3\\xb3','ó').replace('\\xc5\\ba','ź').replace('\\xc5\\xbc','ż')

### Funkcja wyszukująca słowo w Wiktionary

In [18]:
def ret_declin_form_wiktionary(word_, full = True):
    word = ret_lemma(word_)
    url = 'https://en.wiktionary.org/wiki/' + word
    r = requests.get(url)
    tekst = str(r.content)
    
    if 'mianownik (kto? co?)' in tekst:
        start_mian = '<th title="mianownik (kto? co?)'
        start_dop = '<th title="dope\\xc5\\x82niacz (kogo? czego?)'    
        start_cel = '<th title="celownik (komu? czemu?)'
        start_bier = '<th title="biernik (kogo? co?)'
        start_narz = '<th title="narz\\xc4\\x99dnik (kim? czym?)'
        start_miejsc = '<th title="miejscownik (o kim? o czym?)'
        start_wol = '<th title="wo\\xc5\\x82acz (o!)'
        end = '</a>'
        

        mian = tekst.split(start_mian)[1].split(end)[0].split('>')[-1]
        dop = tekst.split(start_dop)[1].split(end)[0].split('>')[-1]
        cel = tekst.split(start_cel)[1].split(end)[0].split('>')[-1]
        bier = tekst.split(start_bier)[1].split(end)[0].split('>')[-1]
        narz = tekst.split(start_narz)[1].split(end)[0].split('>')[-1]
        miejs = tekst.split(start_miejsc)[1].split(end)[0].split('>')[-1]
        wol = tekst.split(start_wol)[1].split(end)[0].split('>')[-1]

        declination_list = [mian, dop, cel, bier, narz, miejs, wol]
        if word_ not in list(map(correct_word,declination_list)):
            
            end = '</a></span>\\n</td></tr>'
            mian = tekst.split(start_mian)[1].split(end)[0].split('>')[-1]
            dop = tekst.split(start_dop)[1].split(end)[0].split('>')[-1]
            cel = tekst.split(start_cel)[1].split(end)[0].split('>')[-1]
            bier = tekst.split(start_bier)[1].split(end)[0].split('>')[-1]
            narz = tekst.split(start_narz)[1].split(end)[0].split('>')[-1]
            miejs = tekst.split(start_miejsc)[1].split(end)[0].split('>')[-1]
            wol = tekst.split(start_wol)[1].split(end)[0].split('>')[-1]
            
            declination_list = [mian, dop, cel, bier, narz, miejs, wol]
            
        declination_list_corr = list(map(correct_word,declination_list))
    
        if full:
            return declination_list_corr
        else:
            #przypadki = ['mianownik', 'dopełniacz', 'celownik', 'biernik', 'narzędnik', 'miejscownik', 'wołacz']
            
            for n,_ in enumerate(declination_list):
                if correct_word(_) == word_:
                    return przypadki[n]
            else:
                return False
            
            return declination_list_corr
    else:
        return False

##### Przykłady działania ww. funkcji

In [803]:
ret_declin_form_wiktionary('księgozbiory', True)

['księgozbiory',
 'księgozbiorów',
 'księgozbiorom',
 'księgozbiory',
 'księgozbiorami',
 'księgozbiorach',
 'księgozbiory']

In [804]:
ret_declin_form_wiktionary('żółw', True)

['żółw', 'żółwia', 'żółwiowi', 'żółwia', 'żółwiem', 'żółwiu', 'żółwiu']

In [805]:
ret_declin_form_wiktionary('przełącznikiem', False)

'narzędnik'

### Funkcja zmieniająca dekliancję słowa na podstawie podanego przypadku 

In [806]:
def change_declination(word, form):
    declin_list = ret_declin_form_wiktionary(word, True)
    declin_form = ret_declin_form_wiktionary(word, False)
    if declin_list != False:
        for n,w in enumerate(declin_list):
            if przypadki[n] == form:
                return str(declin_list[n])
    else:
        return word

# Deklinacja (rozwiązanie lokalne)

In [1]:
import json
rzeczowniki = [json.loads(line) for line in open('nouns.json', 'r')]

In [2]:
def ret_declin_form(word, full = False):
    #jeżeli full = False funkcja zwraca przypadek rzeczownika 
    #jeżeli full = True funkcja zwraca pełną odmianę przez przypadki
            
    lem_szukane_slowo = ret_lemma(word) # słowo w podstawowej formie
    
    
    #wydobądź całą odmianę słowa z lokalnego zbioru rzeczowników
    def wiki_word(lem_word):
        wiki = 'https://en.wiktionary.org/wiki/' 
        wiki_ulr = wiki + change_to(lem_word, 'UTF-8')
        for entry in rzeczowniki:
            if entry['url'] == wiki_ulr:
                return entry
            
    declin_dict = wiki_word(lem_szukane_slowo)
    
    if declin_dict: # jeżeli słownik nie jest pusty
        for k,v in declin_dict['case_forms']['plural'].items():
            if change_to(v, 'unicode') == word:
                return DECLINATION_FORM[k]
        for k,v in declin_dict['case_forms']['singular'].items():
            if change_to(v, 'unicode') == word:
                return DECLINATION_FORM[k]
        return False
    else: # jeżeli jest pusty
        
        return False

##### Przykład

In [10]:
 ret_lemma('Objawami')

'objaw'

In [15]:
ret_declin_form('energii')

'dopełniacz'

In [16]:
ret_declin_form('węgla')

'dopełniacz'

In [19]:
ret_declin_form_wiktionary('krzesła')

['krzesło', 'krzesła', 'krzesłu', 'krzesło', 'krzesłem', 'krześle', 'krzesło']

In [20]:
ret_declin_form_wiktionary('ciepło')

['ciepło', 'ciepła', 'ciepłu', 'ciepło', 'ciepłem', 'cieple', 'ciepło']

In [21]:
ret_declin_form_wiktionary('objawy', True)

['objawy', 'objawów', 'objawom', 'objawy', 'objawami', 'objawach', 'objawy']

### Funkcja odnajdująca słowa podobne do wybranego

##### UWAGA! Proces czasochłonny

In [815]:
def similar_words():
    cand_dict = dict()
    for key in doc: #dla każdego słowa w tekście
        #jeżeli jest to takie samo POS jak wybrane słowo i można dla niego znaleźć wektor
        if nlp(chosen_word)[0].pos_ == key.pos_ and nlp.vocab[str(key)].has_vector:
            score = nlp(str(chosen_word)).similarity(nlp(str(key))) #określ procentową wartość podobieństwa do wybranego słowa
            cand_dict[str(key)] = score # i przypisz temu słowu score
    cand_dict_s = [(k,v) for k,v in sorted(cand_dict.items(), key = lambda item: item[1], reverse = True)] #uporządkuj słowa względem procentowego podobieństwa
    chosen_word_declin = ret_declin_form_wiktionary(chosen_word, False) #sprawdź w jakim przypadku występuje właściwe słowo
    cand_dict_s_new = [(elem[0], elem[1]) for elem in cand_dict_s if different_lemma(chosen_word, elem[0])][:3] # z listy wybranych słów zostaw tylko 3 takie, które
    cand_dict_s_new_words_only = [elem[0] for elem in cand_dict_s_new]
    #mają inną lemmę niż słowo właściwe
    
    if chosen_word_declin != False:
        ans = [change_declination(elem, chosen_word_declin) for elem in cand_dict_s_new_words_only]
        return ans
    else:
        return cand_dict_s_new_words_only

In [816]:
bad_answers = similar_words()
random.shuffle(bad_answers)

In [817]:
bad_answers

['psychopatologii', 'psychoz', 'objawów']

In [818]:
if answers[chosen_word] == tuple():
    answers[chosen_word] = bad_answers

In [819]:
answers

{'zaburzeń': ['psychopatologii', 'psychoz', 'objawów']}

### Wymieszaj odpowiedzi

In [820]:
prep_ans = list(answers.values())[0] + list(answers.keys())
random.shuffle(prep_ans)

### Frontend

In [821]:
import ipywidgets as widgets
button1, button2, button3, button4 = [widgets.Button(description=w) for w in prep_ans]
out = widgets.Output()
def on_button_clicked(b):
    if b.description == chosen_word:
        b.style.button_color = 'lightgreen'
        with out:
            print('Poprawna odpowiedź')
    else:
        b.button_style = 'danger'
        with out:
            print('Błędna odpowiedź')
if (los-1) >=0:
    print(sentences[los-1])
print(replace_word_in_sent(chosen_word, picked_sentence))
button1.on_click(on_button_clicked)
button2.on_click(on_button_clicked)
button3.on_click(on_button_clicked)
button4.on_click(on_button_clicked)
widgets.VBox([button1, button2, button3, button4, out])

Psychoza (gr. psyche – dusza i osis – szaleństwo) – zaburzenie psychiczne definiowane w psychiatrii jako stan umysłu, w którym na skutek zakłóceń w postrzeganiu (percepcji) rzeczywistości, a czasem także _______________ świadomości, osoba ma trudność z odróżnieniem tego, co jest rzeczywiste, a co nie.
